In [1]:
import numpy as np
fpath = '/checkpoint/massran/async_maopy_playground/qp/n{world_size}/{alg}/r{rank}_n{world_size}.npz'

In [2]:
def load_data(world_size=2, alg='gp', rank='0'):
    """ Returns lits of flattened arguments and wallclock-times """
    arg_list, time_list = [], []
    data = np.load(fpath.format(world_size=world_size, alg=alg, rank=rank))['argmin_est'].item()
    for itr in data:
        t, v = data[itr]
        arg_list.append(v.flatten())
        time_list.append(t)
    return time_list, arg_list

In [3]:
def load_objective(fpath='qp_data_sg.npz'):
    """ Returns lambda optimality measure"""
    print('Loading optimality measure...')
    data_fpath = './datasets/' + fpath
    data = np.load(data_fpath)
    a_m = data['A']
    b_v = data['b']
    xstar = data['x_star']
    print('\t Loaded data used to construct measure...', a_m.shape, b_v.shape)
    return lambda x: np.linalg.norm(x - xstar)

In [4]:
def get_world_itr_data(objective, world_size=2, alg='gp'):
    """ Returns (undistilled) x/y data for algorithm/world_size/rank """

    print('Loading all itr-world data (%s) for alg (%s)...' % (world_size, alg))
    time_list, arg_list = [], []
    for rank in range(world_size):
        tl, al = load_data(rank=rank, world_size=world_size, alg=alg)
        time_list.append(tl)
        arg_list.append(al)

    print('Aggregating data...')
    def aggregate(xrank_list):
        """ Aggregate cross-rank list-data across ranks """
        agg_list = []
        for xrank_arg in zip(*xrank_list):
            temp = sum(xrank_arg) / len(xrank_arg)
            agg_list.append(temp)
        return np.array(agg_list)
    arg_list = aggregate(arg_list)

    print('Computing optimlaity...')
    arg_list = np.array(list(map(objective, arg_list)))
    return arg_list

In [5]:
def get_world_time_data(objective, world_size=2, alg='gp', indx=100, step=1., distill=True):
    """ Returns (distilled) x/y data for algorithm/world-size """

    print('Loading all time-world data (%s) for alg (%s)...' % (world_size, alg))
    time_list, arg_list = [], []
    for rank in range(world_size):
        tl, al = load_data(rank=rank, world_size=world_size, alg=alg)
        time_list.append(tl)
        arg_list.append(al)

    def distill_data(t_rank, a_rank):
        """ Keep subset of data to enable async-aggregation across ranks """
        keep_indices = []
        for t_max in x_time:
            # For each t_max, find the single time-index to keep
            i_keep, t_prev = 0, 0
            for i, t in enumerate(t_rank, i_keep):
                if t > t_max:
                    break
                if t_prev <= t:
                    i_keep = i
            keep_indices.append(i_keep)
        temp = list(np.array(a_rank)[keep_indices])
        return temp

    def aggregate(xrank_list):
        """ Aggregate cross-rank list-data across ranks """
        agg_list = []
        for xrank_arg in zip(*xrank_list):
            temp = sum(xrank_arg) / len(xrank_arg)
            agg_list.append(temp)
        return np.array(agg_list)

    if distill:
        print('Distilling data...')
        x_time = [t * step for t in range(indx)]
        arg_time = []
        for tl, al in zip(time_list, arg_list):
            temp = distill_data(tl, al)
            arg_time.append(temp)
    else:
        arg_time = arg_list
        x_time = aggregate(time_list)

    print('Aggregating data...')
    arg_time = aggregate(arg_time)

    print('Computing optimlaity...')
    arg_time = np.array(list(map(objective, arg_time)))
    return x_time, arg_time

In [6]:
objective = load_objective()
print('Done.')

Loading optimality measure...
	 Loaded data used to construct measure... (2560000, 50) (2560000,)
Done.


In [11]:
print('Plotting loading and processing all experiment data')
alg_list = ['agp', 'gp']
world_size_list = [64]
data_dict = {}
for alg in alg_list:
    data_dict[alg] = {}
    for ws in world_size_list:
        data_dict[alg][ws] = {}

        # Load global time data
        data_dict[alg][ws]['g_time'] = []
        data_dict[alg][ws]['g_globj_time'] = []
        tl, yl = get_world_time_data(objective, ws, alg, indx=2000, step=.1)
        data_dict[alg][ws]['g_time'].append(tl)
        data_dict[alg][ws]['g_globj_time'].append(yl)

        # Load global iteration data
        data_dict[alg][ws]['g_globj_itr'] = []
        yl = get_world_itr_data(objective, ws, alg)
        data_dict[alg][ws]['g_globj_itr'].append(yl)
print('Done.')

Plotting loading and processing all experiment data
Loading all time-world data (64) for alg (agp)...
Distilling data...
Aggregating data...
Computing optimlaity...
Loading all itr-world data (64) for alg (agp)...
Aggregating data...
Computing optimlaity...
Loading all time-world data (64) for alg (gp)...
Distilling data...
Aggregating data...
Computing optimlaity...
Loading all itr-world data (64) for alg (gp)...
Aggregating data...
Computing optimlaity...
Done.


In [8]:
print('Initializing bokeh')
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
print('Done.')

Initializing bokeh


Loading BokehJS ...

Done.


In [13]:
print('Plotting data vs. iterations')

num_colors = 10
from bokeh.palettes import Dark2_8 as palette
import itertools  
colors = itertools.cycle(palette)
colors = (c[1] for c in zip(range(num_colors), colors))

p = figure(plot_width=400, plot_height=400,
           title='Average error sequence',
           y_axis_type='log')
p.yaxis.axis_label = 'global loss'
p.xaxis.axis_label = 'iterations'

for alg in data_dict:
    for ws in data_dict[alg]:
        for r in range(len(data_dict[alg][ws]['g_globj_itr'])):
            yl = data_dict[alg][ws]['g_globj_itr'][r]
            xl = np.arange(len(yl))
            p.line(xl, yl, legend='%s:n%s' % (alg, ws), color=next(colors))

show(p)
print('Done.')

Plotting data vs. iterations


Done.


In [12]:
print('Plotting aggregated data vs. time')

num_colors = 10
from bokeh.palettes import Dark2_8 as palette
import itertools
colors = itertools.cycle(palette)
colors = (c[1] for c in zip(range(num_colors), colors))

p = figure(plot_width=400, plot_height=400,
           title='Average error sequence',
           y_axis_type='log')
p.yaxis.axis_label = 'global loss'
p.xaxis.axis_label = 'time (s)'

for alg in data_dict:
    for ws in data_dict[alg]:
        for r in range(len(data_dict[alg][ws]['g_globj_time'])):
            tl = data_dict[alg][ws]['g_time'][r]
            yl = data_dict[alg][ws]['g_globj_time'][r]
            p.line(tl, yl, legend='%s:n%s' % (alg, ws), color=next(colors))

show(p)
print('Done.')

Plotting aggregated data vs. time


Done.
